# Pokemon Classifier Algorithm

Team Starter Pokemon<br>
COGS118B FA21

Alexa Acosta<br>
Lian Lumada<br>
Ramzy Oncy-Avila<br>

[SLIDES](https://docs.google.com/presentation/d/19h5fS61qQCrhrNX4Yu0s71IPTsAPd-4Paam3E0TgaXg/edit?usp=sharing) / VIDEO

## Loading packages

In [1]:
import zipfile
zipfile.ZipFile('data_files.zip').extractall()

In [2]:
import imageio as iio
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import random
import pickle

## Data Import

In [ ]:
# setting some helper variables
directory = "poke_photos/"
pokemon_names = []
pokemon_photos = []

# Uploading our Pokemon images
for filename in os.listdir(directory):
    # Stores names of each pokemon
    name = os.path.splitext(filename)[0]
    pokemon_names.append(name)
    
    # Stores the images of each pokemon - matrix representation
    im = iio.imread(directory + filename, pilmode = "RGBA")[:, :, 0:3] #(120, 120, 3)
    flat_im = [value for rgba in im for value in rgba]
    flatter_im = [value for rgba in flat_im for value in rgba]
    im_final = np.reshape(flatter_im, (14400, 3))
    pokemon_photos.append(im_final)

In [ ]:
# Data frame with corresponding names and image matrix
images = pd.DataFrame(columns = ('Name', 'Image_Mat'))
images['Name'] = pokemon_names
images['Image_Mat'] = pokemon_photos

In [ ]:
# Pokemon info data set
pokemon_data = pd.read_csv('pokemon_to_photos.csv')
pokemon_data = pokemon_data.sort_values("Name", ascending = True).reset_index(drop = True)

## Data Wrangling

In [ ]:
# Sort based on names and reset index
images = images.sort_values("Name", ascending = True).reset_index(drop = True)

In [ ]:
# Combined the two datasets
pokemon_data['Image_Mat'] = images['Image_Mat']

In [ ]:
pokemon_data.head(5)

For each row of `pokemon_data`, a unique Pokemon is associated with it. Each Pokemon has four columns of data attached.
- `Name`: the name of the Pokemon in lowercase
- `Type1`: the primary type of the Pokemon
- `Type2`: the secondary type of the Pokemon, can be NULL
- `Image_Mat`: a tuple storing the RGB values of each pixel (top to bottom, left to right) in the 120x120 Pokemon image.
    - RGB: a value from 0 to 255

## Saving / Loading data

In [3]:
# Save our merged data into a file for easy access later on
#with open('data.pickle', 'wb') as f:
#    pickle.dump(pokemon_data, f)

# Load our merged data into a python variable
with open('data.pickle', 'rb') as f:
    pokemon_data = pickle.load(f)

## Custom Functions

In [4]:
# Function to view the image of the Pokemon
def view_pokemon(val) :
    print("Pokemon: " + pokemon_data['Name'][val])
    pkmn_reverted = np.reshape(pokemon_data['Image_Mat'][val], (120, 120, 3))
    plt.imshow(pkmn_reverted)

In [5]:
# Function to find average RGB value of a Pokemon
def rgb_pkmn(img_mat):
    return np.mean(img_mat, axis = 0)

In [6]:
# Function to get squared distance of each Pokemon's average RGB value from mu of cluster k
def sqDist(img_mat, mu_k):
    rgb_mat = img_mat['Image_Mat']
    sqDmat = np.zeros((np.shape(rgb_mat)[0], np.shape(mu_k)[0]))
    i = 0
    
    for x in rgb_mat:
        for mu in range(len(mu_k)):
            sqDmat[i, mu] = sum((rgb_pkmn(x) - mu_k[mu]) ** 2)
        i += 1
    
    return sqDmat

In [7]:
# Function that returns responsibilities of cluster k
def rnkMat(sqDmat):
    rnk = np.zeros(np.shape(sqDmat))
    mu_col = np.argmin(sqDmat, axis = 1)
    
    for x in range(len(mu_col)):
        rnk[x, mu_col[x]] = 1
    
    return rnk

In [8]:
# Function that calculates new mu of cluster k based on responsibilities
def newMus(img_mat, rnk):
    rgb_mat = img_mat['Image_Mat']
    mu_k = np.zeros((np.shape(rnk)[1], np.shape(rgb_mat[0])[1]))
    
    for k in range(len(rnk[0])):
        x = rgb_mat[rnk[:, k].astype(bool)]
        sum_rnk = len(x)
        sum_x = 0
        if (sum_rnk != 0):
            for pkmn in x:
                sum_x += sum(rgb_pkmn(pkmn))
            mu_k[k,:] = sum_x / sum_rnk
    
    return mu_k

## Training

In [9]:
# get the mean RGB values of all 809 Pokemon in the dataset
mu_pkmn = []

for i in pokemon_data['Image_Mat']:
    mu_pkmn.append(rgb_pkmn(i)) # average rgba values

In [10]:
# select 18 random mus for each type cluster we have to initialize mus
kn = 18 # we have 18 base types
mu_k = np.asarray(random.choices(mu_pkmn, k = kn))

In [11]:
# split data into 80/20 training and testing data sets
training_set = pokemon_data.sample(frac = 0.8, random_state = 500) # set seed to 500 for reproducible results
test_set = pokemon_data.drop(training_set.index)

In [12]:
maxiters = 100;

for iter in range(maxiters):
    sqDmat = sqDist(training_set, mu_k)
    
    rnk = rnkMat(sqDmat)

    mu_old = mu_k
    mu_k = newMus(training_set, rnk)
    
    if sum(abs(mu_old.flatten() - mu_k.flatten())) < 1e-6:
            break

Due to time constraints, we are not able to test the algorithm because it still needed some work to ensure that it was able to compute the `mu_k` matrix properly.